In [1]:
#Basic Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
#Load Dataset
df=pd.read_csv('/home/smriti/Desktop/NLP/MITACS/Anxiety/Data/CSV/Academic/Acad2013.csv')
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
#Combining title and text
df["Text"] = df["Title"] + df["Abstract"]
#Now that we don't need Title or Text, we drop those columns before saving the file
df=df.drop(['Title', 'Abstract'], axis = 1)

/home/smriti/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.head()

,StoreId,documentType,year,Text
0,1511554824,"Journal , Peer Reviewed Journal , Journal Art...",2013,Preceding diagnoses to young adult bipolar dis...
1,1513356585,"Journal , Peer Reviewed Journal , Journal Art...",2013,Cost of anxiety disorders in Japan in 2008: A ...
2,1513357265,"Journal , Peer Reviewed Journal , Journal Art...",2013,Self-harm in young adolescents (12–16 years): ...
3,1513356317,"Journal , Peer Reviewed Journal , Journal Art...",2013,Patterns of risk for anxiety-depression amongs...
4,1513356330,"Journal , Peer Reviewed Journal , Journal Art...",2013,"Functional social support, psychological capit..."


In [6]:
# Convert to list
data=df.Text.values.tolist()
# Remove new line characters
data=[re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data=[re.sub("\'", "", sent) for sent in data]

In [7]:
#Function to clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [8]:
#Clean up text
data_words=list(sent_to_words(data))
print(data_words[:1])

[['preceding', 'diagnoses', 'to', 'young', 'adult', 'bipolar', 'disorder', 'and', 'schizophrenia', 'in', 'nationwide', 'studybackground', 'the', 'aim', 'of', 'this', 'comparative', 'study', 'was', 'to', 'investigate', 'the', 'type', 'and', 'frequency', 'of', 'diagnoses', 'preceding', 'adult', 'bipolar', 'disorder', 'bd', 'and', 'schizophrenia', 'sz', 'methods', 'follow', 'back', 'study', 'of', 'all', 'preceding', 'diagnoses', 'in', 'all', 'patients', 'aged', 'years', 'with', 'primary', 'first', 'time', 'diagnosis', 'of', 'bd', 'or', 'sz', 'in', 'to', 'data', 'were', 'taken', 'from', 'the', 'danish', 'psychiatric', 'central', 'research', 'register', 'dpcrr', 'including', 'icd', 'and', 'icd', 'diagnoses', 'results', 'the', 'numbers', 'of', 'patients', 'with', 'any', 'preceding', 'diagnoses', 'amounted', 'to', 'in', 'bd', 'and', 'in', 'sz', 'with', 'affective', 'disorders', 'excluding', 'bd', 'being', 'the', 'most', 'frequent', 'preceding', 'diagnosis', 'vs', 'followed', 'by', 'psychoses'

In [9]:
# Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['preceding', 'diagnoses', 'to', 'young', 'adult', 'bipolar', 'disorder', 'and', 'schizophrenia', 'in', 'nationwide', 'studybackground', 'the', 'aim', 'of', 'this', 'comparative', 'study', 'was', 'to', 'investigate', 'the', 'type', 'and', 'frequency', 'of', 'diagnoses', 'preceding', 'adult', 'bipolar', 'disorder', 'bd', 'and', 'schizophrenia', 'sz', 'methods', 'follow', 'back', 'study', 'of', 'all', 'preceding', 'diagnoses', 'in', 'all', 'patients', 'aged_years', 'with', 'primary', 'first', 'time', 'diagnosis', 'of', 'bd', 'or', 'sz', 'in', 'to', 'data', 'were', 'taken', 'from', 'the', 'danish', 'psychiatric', 'central', 'research', 'register', 'dpcrr', 'including', 'icd', 'and', 'icd', 'diagnoses', 'results', 'the', 'numbers', 'of', 'patients', 'with', 'any', 'preceding', 'diagnoses', 'amounted', 'to', 'in', 'bd', 'and', 'in', 'sz', 'with', 'affective', 'disorders', 'excluding', 'bd', 'being', 'the', 'most', 'frequent', 'preceding', 'diagnosis', 'vs', 'followed', 'by', 'psychoses', 'p

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

# Create Dictionary needed for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=55,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

[['precede', 'diagnose', 'young', 'adult', 'bipolar', 'disorder', 'study', 'investigate', 'type', 'frequency', 'diagnosis', 'precede', 'adult', 'bipolar', 'disorder', 'method', 'follow', 'back', 'study', 'precede', 'diagnosis', 'patient', 'primary', 'first', 'time', 'diagnosis', 'bd', 'datum', 'take', 'danish', 'psychiatric', 'include', 'diagnose', 'result', 'number', 'patient', 'precede', 'diagnosis', 'amount', 'affective', 'disorder', 'exclude', 'frequent', 'precede', 'diagnosis', 'follow', 'psychosis', 'substance', 'disorder', 'severe', 'stress', 'equally', 'frequent', 'sample', 'personality', 'disorder', 'rate', 'conduct', 'disorder', 'rather', 'low', 'sample', 'precede', 'diagnosis', 'onset', 'adolescence', 'overall', 'patient', 'statistically', 'significant', 'early', 'onset', 'psychiatric', 'disorder', 'compare', 'mean', 'age', 'lt', 'regression', 'analysis', 'indicate', 'associate', 'increased', 'risk', 'experience', 'precede', 'affective', 'disorder', 'increase', 'risk', 'prec

In [15]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(37,
  '0.000*"depressive" + 0.000*"study" + 0.000*"symptom" + 0.000*"base" + '
  '0.000*"help" + 0.000*"employee" + 0.000*"company" + 0.000*"work" + '
  '0.000*"course" + 0.000*"care"'),
 (19,
  '0.000*"self" + 0.000*"harm" + 0.000*"thought" + 0.000*"young" + '
  '0.000*"year" + 0.000*"depression" + 0.000*"developmental" + '
  '0.000*"adolescent" + 0.000*"report" + 0.000*"trajectory"'),
 (45,
  '0.000*"treatment" + 0.000*"impact" + 0.000*"legal" + 0.000*"status" + '
  '0.000*"child" + 0.000*"refugee" + 0.000*"asylum_seeker" + 0.000*"group" + '
  '0.000*"outcome" + 0.000*"development"'),
 (0,
  '0.000*"disorder" + 0.000*"study" + 0.000*"treatment" + 0.000*"mental" + '
  '0.000*"help" + 0.000*"significant" + 0.000*"eat" + 0.000*"result" + '
  '0.000*"anxiety" + 0.000*"base"'),
 (20,
  '0.000*"patient" + 0.000*"cancer" + 0.000*"distress" + 0.000*"peritraumatic" '
  '+ 0.000*"assess" + 0.000*"psychological" + 0.000*"high" + 0.000*"predictor" '
  '+ 0.000*"psychiatric" + 0.000*"study"'),


In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -12.41702310609058


In [17]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3681821560584694


In [18]:
num_topics = 55
topic_words = []
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic_words.append([id2word[pair[0]] for pair in tt])

In [19]:
for i in range(0,55):
    print(topic_words[i])

['disorder', 'study', 'treatment', 'mental', 'help', 'significant', 'eat', 'result', 'anxiety', 'base']
['social', 'internalize', 'anxiety', 'subject', 'skill', 'impairment', 'independent', 'symptom', 'interaction', 'co']
['anxiety', 'associate', 'self', 'risk', 'high', 'low', 'smoking', 'seek', 'adolescent', 'level']
['impulsivity', 'behavioral', 'educational', 'exhibit', 'prior', 'training', 'measure', 'unclear', 'expression', 'background']
['depression', 'cognitive', 'risk', 'woman', 'major', 'study', 'diagnostic', 'mental', 'report', 'specificity']
['cbm', 'recall', 'drive', 'training', 'interpretation', 'may', 'present', 'involve', 'early', 'long_lasting']
['refer', 'minor', 'sufficient', 'duloxetine', 'enable', 'relief', 'collaborative_care', 'major', 'patient', 'depression']
['trend', 'hypnotic', 'treating', 'sleep', 'diary', 'daytime', 'med', 'elevate', 'prove', 'volunteer']
['maternal', 'paternal', 'emotional', 'predictor', 'child', 'anxiety', 'depression', 'behavioral', 'reac

In [24]:
df=pd.DataFrame({'Year':[],'Source':[],'Topic_ID':[],'Most_freq_words':[]})
df.head()

,Most_freq_words,Source,Topic_ID,Year


In [25]:
df['Most_freq_words']=topic_words

In [26]:
df = df.assign(Year='2013')
df = df.assign(Source='Academic')
df.shape

(55, 4)

In [27]:
ls=[]
for i in range(0,55):
    ls.append(i)
df['Topic_ID']=ls

In [28]:
df.head()

,Most_freq_words,Source,Topic_ID,Year
0,"[disorder, study, treatment, mental, help, sig...",Academic,0,2013
1,"[social, internalize, anxiety, subject, skill,...",Academic,1,2013
2,"[anxiety, associate, self, risk, high, low, sm...",Academic,2,2013
3,"[impulsivity, behavioral, educational, exhibit...",Academic,3,2013
4,"[depression, cognitive, risk, woman, major, st...",Academic,4,2013


In [30]:
df.to_csv("topic_words_j2013.csv")